In [ ]:
!pip install tweet-preprocessor
!pip install textblob
!pip install sastrawi
!pip install emoji
!pip install PySastrawi

In [ ]:
import os
import pandas as pd
import tweepy
import re
import string
from textblob import TextBlob
import preprocessor as p
from preprocessor.api import clean, tokenize, parse
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import datetime
from datetime import timedelta
import numpy as np
import emoji
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [ ]:
import pandas as pd
def load_data():
  data = pd.read_csv('PILGUB.csv')
  return data

In [ ]:
tweet_df = load_data()
tweet_df

In [ ]:
#cleaning
def remove_pattern(text, pattern_regex):
  r = re.findall(pattern_regex, text)
  for i in r:
    text = re.sub(i, '', text)
  return text

In [ ]:
tweet_df['clean_tweet'] = np.vectorize(remove_pattern)(tweet_df['Komentar'], " *RT* | *@[\w]*")
tweet_df.head(10)

In [ ]:
#remove emoji
def remove(text):
  text =' '.join(re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ", text).split())
  return text
tweet_df['remove_http'] = tweet_df['Komentar'].apply(lambda x: remove(x))
tweet_df.head(10)

In [ ]:
#remove
def remov(text):
  text = re.sub(r'r\$\w*', '', text)
  text = re.sub(r'^RT[\s]+', '', text)
  text = re.sub(r'#', '', text)
  text = re.sub(r'[0-9]+', '',text)

  return text

tweet_df['removing_hastag'] = tweet_df['remove_http'].apply(lambda x: remov(x))
tweet_df.head(10)

In [ ]:
#remove duplikat
tweet_df.drop_duplicates(subset = "removing_hastag", keep = 'first', inplace = True)
tweet_df.head()

In [ ]:
# Fungsi case folding
def case_folding(text):
    return text.lower()

# Menerapkan ke kolom DataFrame
tweet_df['case_folding'] = tweet_df['removing_hastag'].apply(case_folding)

# Cek hasil
tweet_df[['removing_hastag', 'case_folding']].head()

In [ ]:
# Import library yang diperlukan untuk stopword
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords_indonesia = stopwords.words('indonesian')

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary

# Ambil stopwords dari Sastrawi
stop_factory = StopWordRemoverFactory().get_stop_words()

# Tambahan stopwords gaul/custom
more_stopwords = [
    'yg', 'utk', 'cuman', 'deh', 'btw', 'tapi', 'gua', 'gue', 'lo', 'lu',
    'kalo', 'trs', 'jd', 'nih', 'ntr', 'nya', 'lg', 'gk', 'ecusli', 'dpt',
    'dr', 'kpn', 'kok', 'kyk', 'donk', 'yah', 'u', 'ya', 'ga', 'km', 'eh',
    'sih', 'bang', 'br', 'rp', 'jt', 'kan', 'gpp', 'sm', 'usah',
    'mas', 'sob', 'thx', 'ato', 'jg', 'gw', 'wkwkwk', 'mak', 'haha', 'iy', 'k',
    'tp', 'haha', 'dg', 'dri', 'duh', 'ye', 'wkwk', 'syg',
    'nerjemahin', 'gaes', 'guys', 'moga', 'kmrn', 'nemu', 'yukk',
    'klas', 'iw', 'ew', 'lho', 'sbnry', 'org', 'gtu', 'bwt',
    'krlga', 'clau', 'lbh', 'cpet', 'ku', 'wke', 'mba', 'sdh', 'oi',
    'spt', 'dlm', 'bs', 'krn', 'jgn', 'sapa', 'sh', 'wakakaka',
    'sihhh', 'hehe', 'ih', 'dgn', 'la', 'kl', 'ttg', 'mana', 'kmna', 'kmn',
    'tdk', 'tuh', 'dah', 'kek', 'ko', 'pls', 'bbrp', 'pd', 'mah', 'dhhh',
    'kpd', 'kzl', 'byar', 'si', 'sii', 'cm', 'sy', 'hahahaha', 'weh',
    'dlu', 'tuhh', 'guweh', 'amp'
]

# Gabungkan semua stopwords
combined_stopwords = stop_factory + more_stopwords

# Buat dictionary & stopword remover
dictionary = ArrayDictionary(combined_stopwords)
stopword_remover = StopWordRemover(dictionary)

# Fungsi untuk melakukan stopword removal
def remove_stopwords(text):
    return stopword_remover.remove(text)

# Terapkan fungsi ke DataFrame dan simpan ke kolom baru
tweet_df['stopword_removed'] = tweet_df['case_folding'].apply(lambda x: remove_stopwords(x))

# Tampilkan hasil data terbaru
print(tweet_df[['case_folding', 'stopword_removed']].head(10))

In [ ]:
tweet_df.head()

In [ ]:
# Import library yang dibutuhkan
import pandas as pd
import nltk

# Download the 'punkt' and 'punkt_tab' resources
nltk.download('punkt')
nltk.download('punkt_tab')

from nltk.tokenize import word_tokenize

# Fungsi untuk melakukan tokenizing (mengubah teks jadi list kata)
def tokenize_text(text):
    return word_tokenize(text)

# Terapkan tokenizing dan simpan ke kolom baru
tweet_df['tokenized'] = tweet_df['stopword_removed'].apply(lambda x: tokenize_text(x))

# Konversi hasil token ke string dengan tanda kutip
tweet_df['tokenized'] = tweet_df['tokenized'].apply(lambda tokens: "[" + ", ".join([repr(token) for token in tokens]) + "]")

# Tampilkan hasil data terbaru
print(tweet_df[['stopword_removed', 'tokenized']].head())

In [ ]:
tweet_df.head()

In [ ]:
# Import library yang dibutuhkan untuk stemming
import pandas as pd
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Buat stemmer dari Sastrawi
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Fungsi stemming
def stem_text(text):
    return stemmer.stem(text)

# Simpan hasil stemming ke kolom baru
tweet_df['stemmed'] = tweet_df['tokenized'].apply(lambda x: stem_text(x))

# Tampilkan hasil data terbaru
print(tweet_df[['tokenized', 'stemmed']])


In [ ]:
tweet_df.head()

In [ ]:
#simpan data bersih
tweet_df.to_csv('stemming.csv', encoding ='utf8', index = False)